# Orientación de las calles en las ciudades colombianas

A partir del <a href=https://github.com/gboeing/osmnx-examples/blob/master/notebooks/17-street-network-orientations.ipynb><strong>ejemplo original</strong></a> publicado por <a href=https://github.com/gboeing><strong>Geoff Boeing</strong></a>, creador de la librería OSMNX, se me ocurrió realizar un ejercicio rápido para analizar de forma descriptiva el trazado urbano de las ciudades de Colombia y de forma implicita pobrar la librería mencionada.

## Prerrequisitos

#### Tener instalado Anaconda

Si bien no es indispensable, para seguir este tutorial recomiendo instalar el IDE <a href=https://www.anaconda.com/distribution/><strong>Anaconda</strong></a>

#### Librerías necesarias

Adicionalmente es requerido tener instaladas la siguientes librerías:
- numpy
- pandas

#### Instalar OSMNX

Esto se hace desde una ventana de comandos (con permisos de administrador) con la siguiente instrucción:

conda install -c conda-forge osmnx

## Cargue de librerías y validación de la versión de OMNX

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import osmnx as ox
import pandas as pd

ox.config(log_console=True, use_cache=True)
weight_by_length = False

ox.__version__

'0.10'

## Definición del listado de ciudades a consultar

Se debe crear una lista en la que el label será el nombre de la ciudad y el valor el query a consultar en Open Street Maps. Es indispensable asegurar previamente que el query retorna un objeto de tipo POLYGON. Para el caso de Colombia muchos de los querys que permiten consultar las fronteras de las ciudades no retonran el tipo de dato adecuado para este API, así que usaré un método alternativo que consiste en consultar las calles que se encuentran a cierta distancia desde un punto determinado.

In [2]:
places = {'Barranquilla'  : 'Barranquilla, Atlántico, Colombia',
          'Bogotá'        : 'Bogota, Bogota Capital District, Colombia',
          'Manizales'     : 'Manizales, Centrosur, Caldas, Colombia',
          'Florencia'     : 'Florencia, Caquetá, Colombia',
          'Popayán'       : 'Popayán, Cauca, Colombia',
          'Valledupar'    : 'Valledupar, Cesar, Colombia',
          'Quibdó'        : 'Quibdó, Chocó, Colombia',
          'Montería'      : 'Montería, Córdoba, Colombia',
          'Neiva'         : 'Neiva, Huila, Colombia',
          'Riohacha'      : 'Riohacha, La Guajira, Colombia', 
          'Villavicencio' : 'Villavicencio, Meta, Colombia',
          'Pasto'         : 'Pasto, Nariño, Colombia',
          'Mocoa'         : 'Mocoa, Putumayo, 860001, Colombia', 
          'Armenia'       : 'Armenia, Quindío, Colombia', 
          'Pereira'       : 'Pereira, Risaralda, Colombia',
          'Bucaramanga'   : 'Bucaramanga, Santander, Colombia', 
          'Sincelejo'     : 'Sincelejo, Sucre, Colombia',
          'Ibagué'        : 'Ibagué, Tolima, Colombia',
          'Cali'          : 'Cali, Valle del Cauca, Colombia',
          'Medellín'      : 'Medellín, Zona Urbana Medellín, Medellín, Valle de Aburrá, Antioquia, 0500, Colombia',
          'Tunja'         : 'Tunja, Boyacá, 15001000, Colombia',
          'Mitú'          : 'Mitú, Vaupés, Colombia',
          'Arauca'        : 'Arauca, 80001, Colombia',
          'Cartagena'     : 'Cartagena, Cartagena de Indias, Bolívar, 130015, Colombia',
          'Yopal'         : 'Yopal, Casanare, 850001, Colombia',
          'San José G.'   : 'San José del Guaviare, Guaviare, 95001, Colombia',
          'Cúcuta'        : 'Cúcuta, Norte de Santander, 047, Colombia',
          'Puerto Carreño': 'Puerto Carreño, Vichada, 990001-990017, Colombia',
          'Santa Marta'   : 'Santa Marta, Magdalena, 575, Colombia'
          #'San Andrés'    : 'San Andrés, Archipiélago de San Andrés, Providencia y Santa Catalina, 123, Colombia'
          #'Inirida'       : 'Inírida, Guainía, 940001, Colombia'
          #'Leticia'       : 'Leticia, Amazonas, Colombia'
         }

### Validación del tipo de objetos retornados

In [3]:
# Retorna un GeoDataFrame a partir de una lista de valores "query" a consultar en OSM
# Como ya se mencionó, para el caso de este ejemplo esta sección no se utiliza

#gdf = ox.gdf_from_places(places.values())
#gdf

En caso que alguno de los objetos retornados no sean POLYGON es necesario eliminar esta ciudad de la lista.

### Definición de la función que determina la orientación opuesta de una dirección

In [4]:
# Obtiene la orientación contrario para un valor en grados
def reverse_bearing(x):
    return x + 180 if x < 180 else x - 180

In [5]:
# Creación de los pares de valores que indican la magnitud de calles en una dirección y su opuesto
bearings = {}
for place in sorted(places.keys()):
    
    # Obtiene un GeoDataFrame para un único query de consulta en OSM
    query = places[place]
    
    # Para las consultas que retornan un objeto POLYGON se utilizaría la siguiente instrucción:
    #G = ox.graph_from_place(query, network_type='drive')
    
    # Para consultar a partir de un punto se utiliza (5K metros desde el punto de consulta):
    G = ox.graph_from_address(query, distance=5000, network_type='drive')
    
    # Primero convierte el grafo dirgido de los bordes del polígono en un grafo no dirigido
    # A continuación agrega la orientación de cada punto del borde como un atributo
    Gu = ox.add_edge_bearings(ox.get_undirected(G))
    
    # Por defecto esta variable se inicializa en False
    if weight_by_length:
        # Crea un estructura con las direcciones de los bordes y sus opuestos
        city_bearings = []
        for u, v, k, d in Gu.edges(keys=True, data=True):
            city_bearings.extend([d['bearing']] * int(d['length']))
        b = pd.Series(city_bearings)
        bearings[place] = pd.concat([b, b.map(reverse_bearing)]).reset_index(drop='True')
    else:
        # Crea un estructura con las direcciones de los bordes y sus opuestos
        b = pd.Series([d['bearing'] for u, v, k, d in Gu.edges(keys=True, data=True)])
        bearings[place] = pd.concat([b, b.map(reverse_bearing)]).reset_index(drop='True')

In [6]:
# Ajusta los valores con orientaciones extremas
def count_and_merge(n, bearings):
    # Asignación de los valores opuestos para evitar redundancia de valores en 0° y 360°
    n = n * 2
    bins = np.arange(n + 1) * 360 / n
    count, _ = np.histogram(bearings, bins=bins)
    
    # Agrupación de los valores límites
    count = np.roll(count, 1)
    return count[::2] + count[1::2]

### Función que dibuja un plano polar para las orientaciones de las calles

In [7]:
# Crea el mapa polar para cada una de las ubicaciones
def polar_plot(ax, bearings, n=36, title=''):

    bins = np.arange(n + 1) * 360 / n
    count = count_and_merge(n, bearings)
    _, division = np.histogram(bearings, bins=bins)
    frequency = count / count.sum()
    division = division[0:-1]
    width =  2 * np.pi / n

    ax.set_theta_zero_location('N')
    ax.set_theta_direction('clockwise')

    x = division * np.pi / 180
    bars = ax.bar(x, height=frequency, width=width, align='center', bottom=0, zorder=2,
                  color='#003366', edgecolor='k', linewidth=0.5, alpha=0.7)
    
    ax.set_ylim(top=frequency.max())
    
    title_font = {'family':'sans-serif', 'size':24, 'weight':'bold'}
    xtick_font = {'family':'sans-serif', 'size':10, 'weight':'bold', 'alpha':1.0, 'zorder':3}
    ytick_font = {'family':'sans-serif', 'size': 9, 'weight':'bold', 'alpha':0.2, 'zorder':3}
    
    ax.set_title(title.upper(), y=1.05, fontdict=title_font)
    
    ax.set_yticks(np.linspace(0, max(ax.get_ylim()), 5))
    yticklabels = ['{:.2f}'.format(y) for y in ax.get_yticks()]
    yticklabels[0] = ''
    ax.set_yticklabels(labels=yticklabels, fontdict=ytick_font)
    
    xticklabels = ['N', '', 'E', '', 'S', '', 'W', '']
    ax.set_xticklabels(labels=xticklabels, fontdict=xtick_font)
    ax.tick_params(axis='x', which='major', pad=-2)

In [8]:
# Crear el plano y los ejes a partir de la cantidad de lugares
n = len(places)
ncols = int(np.ceil(np.sqrt(n)))
nrows = int(np.ceil(n / ncols))
figsize = (ncols * 5, nrows * 5)
fig, axes = plt.subplots(nrows, ncols, figsize=figsize, subplot_kw={'projection':'polar'})

# Dibujar el histograma polar de cada ciudad
for ax, place in zip(axes.flat, sorted(places.keys())):
    polar_plot(ax, bearings[place].dropna(), title=place)

# Agregar el título y salva la imagen
suptitle_font = {'family':'sans-serif', 'fontsize':60, 'fontweight':'normal', 'y':1.07}
fig.suptitle('Orientación de las calles', **suptitle_font)
fig.tight_layout()
fig.subplots_adjust(hspace=0.35)
fig.savefig('images/street-orientations.png', dpi=120, bbox_inches='tight')
plt.close()